In [1]:
%matplotlib inline

import numpy as np
import pandas as pd

### 用户支付数据

In [2]:
df_pay = pd.read_csv('../data/user_pay.txt',header=None)
df_pay.columns = ['user_id', 'shop_id', 'timestamp']
df_pay.head()

,user_id,shop_id,timestamp
0,22127870,1862,2015-12-25 17:00:00
1,3434231,1862,2016-10-05 11:00:00
2,16955285,1862,2016-02-10 15:00:00
3,13799128,1862,2016-01-13 14:00:00
4,13799128,1862,2016-07-05 12:00:00


In [3]:
df_pay['date'] = df_pay['timestamp'].apply(lambda x: x[:10])
df_pay = df_pay[(df_pay['date'] >= '2016-09-06') &
                (df_pay['date'] <= '2016-10-31')]
df_pay = df_pay[(df_pay['date'] > '2016-10-07') |
                (df_pay['date'] < '2016-10-01')]
df_pay.head()

,user_id,shop_id,timestamp,date
5,20244878,1862,2016-09-17 15:00:00,2016-09-17
11,13211167,1862,2016-09-29 20:00:00,2016-09-29
17,17609360,1862,2016-10-15 17:00:00,2016-10-15
19,15262746,1862,2016-10-18 19:00:00,2016-10-18
24,11775594,1862,2016-09-25 17:00:00,2016-09-25


In [4]:
df_agg_pay = df_pay.groupby(['shop_id', 'date']).agg({
    'timestamp': ['min', 'max', 'size'],
}).reset_index()
df_agg_pay.columns = ['shop_id', 'date', 'min_ts', 'max_ts', 'pay_count']
df_agg_pay.head()

,shop_id,date,min_ts,max_ts,pay_count
0,1,2016-09-06,2016-09-06 07:00:00,2016-09-06 22:00:00,262
1,1,2016-09-07,2016-09-07 07:00:00,2016-09-07 22:00:00,251
2,1,2016-09-08,2016-09-08 07:00:00,2016-09-08 22:00:00,283
3,1,2016-09-09,2016-09-09 07:00:00,2016-09-09 22:00:00,274
4,1,2016-09-10,2016-09-10 07:00:00,2016-09-10 22:00:00,252


In [5]:
df_agg_pay['operate_hour'] = ((pd.to_datetime(
    df_agg_pay['max_ts'])-pd.to_datetime(df_agg_pay['min_ts'])) / np.timedelta64(1, 'h')).astype(int)
df_agg_pay= df_agg_pay.drop(['min_ts','max_ts'],axis=1)
df_agg_pay.head()

,shop_id,date,pay_count,operate_hour
0,1,2016-09-06,262,15
1,1,2016-09-07,251,15
2,1,2016-09-08,283,15
3,1,2016-09-09,274,15
4,1,2016-09-10,252,15


### 处理用户浏览数据

In [6]:
df_browser = pd.read_csv('../data/user_view.txt')
df_browser.columns = ['user_id', 'shop_id', 'timestamp']

df_browser['date'] = df_browser['timestamp'].apply(lambda x: x[:10])
df_browser = df_browser[(df_browser['date'] >= '2016-09-06') & (df_browser['date'] <= '2016-10-31')]
df_browser = df_browser[(df_browser['date'] > '2016-10-07') | (df_browser['date'] < '2016-10-01')]
browser_stat_day = df_browser.groupby(['shop_id', 'date']).size().reset_index()
browser_stat_day.columns = ['shop_id', 'date', 'browser_count']
browser_stat_day.head()

,shop_id,date,browser_count
0,1,2016-09-06,102
1,1,2016-09-07,66
2,1,2016-09-08,100
3,1,2016-09-09,90
4,1,2016-09-10,85


### 构造日期维度表

In [7]:
dim_list = []
shop_id = df_agg_pay['shop_id'].unique()
date_list = df_agg_pay['date'].unique()
for _id in shop_id:
    for date in date_list:
        data = {'shop_id':_id,'date':date}
        dim_list.append(data)
df_dim = pd.DataFrame(dim_list)
df_dim.head()

,shop_id,date
0,1,2016-09-06
1,1,2016-09-07
2,1,2016-09-08
3,1,2016-09-09
4,1,2016-09-10


In [8]:
df = pd.merge(df_dim,df_agg_pay,how='left').fillna(0)
df = pd.merge(df,browser_stat_day,how='left').fillna(0)
df.head()

,shop_id,date,pay_count,operate_hour,browser_count
0,1,2016-09-06,262.0,15.0,102.0
1,1,2016-09-07,251.0,15.0,66.0
2,1,2016-09-08,283.0,15.0,100.0
3,1,2016-09-09,274.0,15.0,90.0
4,1,2016-09-10,252.0,15.0,85.0


In [9]:
df['dayofweek'] = pd.to_datetime(df['date']).dt.dayofweek
# 需要处理节假日及调休

def deal_holiday(x):
    result = 1 if pd.to_datetime(x).strftime("%w") in ['0', '6'] else 0
    if x in ['2016-09-15','2016-09-16','2016-09-17']:
        result=1
    if x in ['2016-09-18','2016-10-08','2016-10-09']:
        result=0
    return result

df['is_holiday'] = df['date'].apply(deal_holiday)
day_week_config = {
'2016-09-06':1,
'2016-09-07':1,
'2016-09-08':1,
'2016-09-09':1,
'2016-09-10':1,
'2016-09-11':1,
'2016-09-12':1,

'2016-09-13':2,
'2016-09-14':2,
'2016-09-15':2,
'2016-09-16':2,
'2016-09-17':2,
'2016-09-18':2,
'2016-09-19':2,

'2016-09-20':3,
'2016-09-21':3,
'2016-09-22':3,
'2016-09-23':3,
'2016-09-24':3,
'2016-09-25':3,
'2016-09-26':3,

'2016-09-27':4,
'2016-09-28':4,
'2016-09-29':4,
'2016-09-30':4,
'2016-10-08':4,
'2016-10-09':4,
'2016-10-10':4,

'2016-10-11':5,
'2016-10-12':5,
'2016-10-13':5,
'2016-10-14':5,
'2016-10-15':5,
'2016-10-16':5,
'2016-10-17':5,

'2016-10-18':6,
'2016-10-19':6,
'2016-10-20':6,
'2016-10-21':6,
'2016-10-22':6,
'2016-10-23':6,
'2016-10-24':6,

'2016-10-25':7,
'2016-10-26':7,
'2016-10-27':7,
'2016-10-28':7,
'2016-10-29':7,
'2016-10-30':7,
'2016-10-31':7
}
df['week_id'] = df['date'].apply(lambda x:day_week_config.get(x))
df.head()

,shop_id,date,pay_count,operate_hour,browser_count,dayofweek,is_holiday,week_id
0,1,2016-09-06,262.0,15.0,102.0,1,0,1
1,1,2016-09-07,251.0,15.0,66.0,2,0,1
2,1,2016-09-08,283.0,15.0,100.0,3,0,1
3,1,2016-09-09,274.0,15.0,90.0,4,0,1
4,1,2016-09-10,252.0,15.0,85.0,5,1,1


In [10]:
for column in ['pay_count','operate_hour','browser_count','dayofweek','is_holiday']:
    df[column]=df[column].astype(int)

In [11]:
df.to_csv('../data/pay_stat_day.csv',index=False)

In [12]:
df.shape

(98000, 8)

In [13]:
df.head()

,shop_id,date,pay_count,operate_hour,browser_count,dayofweek,is_holiday,week_id
0,1,2016-09-06,262,15,102,1,0,1
1,1,2016-09-07,251,15,66,2,0,1
2,1,2016-09-08,283,15,100,3,0,1
3,1,2016-09-09,274,15,90,4,0,1
4,1,2016-09-10,252,15,85,5,1,1
